In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [4]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

In [1]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./df_contents.csv')
df.head()

,Unnamed: 0,title,review
0,0,우리들의 블루스,이 화려한 캐스팅 라인업을 보고 안 볼수가 있나.. ㄷㄷㄷ 게다가 노희경 극본..ㄷ...
1,1,우리들의 블루스,"침묵의 순간들이 더욱 처연하게 빛나는, 노희경 극본의 또 다른 경지."
2,2,우리들의 블루스,세상에..노희경 작가와 이 배우들이라면 어떤 스토리가 됐든 꼭 보고싶다..
3,3,우리들의 블루스,하 진짜 좋아하는 배우 전부 다 나오네…\r\n.\r\n한수의 위험한 생각이 뭔지 ...
4,4,우리들의 블루스,이 작품 뒤에 결혼할것 같은 두 찐커플이 출연


In [5]:
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

In [6]:
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    nouns = [s for s, t in tagged if t in ['NNG', 'NNP'] and len(s) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab()
    processed_data = []
    for sent in tqdm(df['review']):
        sentence = clean_text(str(sent).replace("\n", "").strip())
        processed_data.append(get_nouns(tokenizer, sentence))
    return processed_data

In [7]:
preprocessed_data = tokenize(df)
preprocessed_data[:5]

100%|██████████████████████████████████| 25676/25676 [00:14<00:00, 1780.80it/s]


[['캐스팅', '라인업', '노희경', '극본', '몰입', '차승원'],
 ['침묵', '순간', '노희경', '극본', '경지'],
 ['세상', '노희경', '작가', '배우', '스토리'],
 ['배우', '한수', '위험', '생각', '고통'],
 ['작품', '결혼', '커플', '출연']]

In [8]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
dictionary = corpora.Dictionary(preprocessed_data)

2022-04-15 18:18:11,785 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2022-04-15 18:18:12,440 : INFO : adding document #10000 to Dictionary(10735 unique tokens: ['극본', '노희경', '라인업', '몰입', '차승원']...)
2022-04-15 18:18:12,967 : INFO : adding document #20000 to Dictionary(14872 unique tokens: ['극본', '노희경', '라인업', '몰입', '차승원']...)
2022-04-15 18:18:13,215 : INFO : built Dictionary(16060 unique tokens: ['극본', '노희경', '라인업', '몰입', '차승원']...) from 25676 documents (total 275668 corpus positions)
2022-04-15 18:18:13,217 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(16060 unique tokens: ['극본', '노희경', '라인업', '몰입', '차승원']...) from 25676 documents (total 275668 corpus positions)", 'datetime': '2022-04-15T18:18:13.217831', 'gensim': '4.1.2', 'python': '3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-8.1-6.3.9600-SP0', 'event': 'created'}


In [11]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

2022-04-15 18:18:50,710 : INFO : discarding 7032 tokens: [('강세', 1), ('진에', 1), ('우빈', 1), ('파라다이스', 1), ('후감', 1), ('김규태', 1), ('프랜즈', 1), ('다저', 1), ('코빅', 1), ('돈때', 1)]...
2022-04-15 18:18:50,713 : INFO : keeping 9028 tokens which were in no less than 2 and no more than 12838 (=50.0%) documents
2022-04-15 18:18:50,744 : INFO : resulting dictionary: Dictionary(9028 unique tokens: ['극본', '노희경', '라인업', '몰입', '차승원']...)


In [13]:
corpus = [dictionary.doc2bow(text) for text in preprocessed_data]

In [14]:
num_topics = 4
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=1
)

2022-04-15 18:19:34,991 : INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
2022-04-15 18:19:34,999 : INFO : using serial LDA version on this node
2022-04-15 18:19:35,014 : INFO : running online (multi-pass) LDA training, 4 topics, 20 passes over the supplied corpus of 25676 documents, updating model once every 2000 documents, evaluating perplexity every 2000 documents, iterating 400x with a convergence threshold of 0.001000
2022-04-15 18:19:37,884 : INFO : -9.821 per-word bound, 904.4 perplexity estimate based on a held-out corpus of 2000 documents with 20937 words
2022-04-15 18:19:37,886 : INFO : PROGRESS: pass 0, at document #2000/25676
2022-04-15 18:19:40,284 : INFO : optimized alpha [0.20886348, 0.20059188, 0.19753723, 0.17163032]
2022-04-15 18:19:40,291 : INFO : merging changes from 2000 documents into a model of 25676 documents
2022-04-15 18:19:40,299 : INFO : topic #0 (0.209): 0.029*"사랑" + 0.022*"배우" + 0.015*"드라마" + 0.014*"연기" + 0.012*"작가" + 0.012*"청춘" + 0.01

2022-04-15 18:19:57,999 : INFO : topic #1 (0.193): 0.027*"사람" + 0.022*"학교" + 0.020*"생각" + 0.013*"드라마" + 0.010*"시간" + 0.010*"사회" + 0.009*"이야기" + 0.009*"학생" + 0.008*"폭력" + 0.007*"필요"
2022-04-15 18:19:58,003 : INFO : topic #2 (0.266): 0.060*"드라마" + 0.021*"연기" + 0.016*"소희" + 0.016*"연출" + 0.015*"원작" + 0.015*"웹툰" + 0.015*"배우" + 0.014*"장면" + 0.012*"장르" + 0.009*"정도"
2022-04-15 18:19:58,006 : INFO : topic #3 (0.188): 0.019*"신파" + 0.016*"캐릭터" + 0.015*"게임" + 0.015*"작품" + 0.011*"액션" + 0.010*"영화" + 0.010*"한국" + 0.010*"드라마" + 0.009*"정도" + 0.008*"장면"
2022-04-15 18:19:58,009 : INFO : topic diff=0.456954, rho=0.377964
2022-04-15 18:19:59,566 : INFO : -8.153 per-word bound, 284.6 perplexity estimate based on a held-out corpus of 2000 documents with 22102 words
2022-04-15 18:19:59,567 : INFO : PROGRESS: pass 0, at document #16000/25676
2022-04-15 18:20:00,947 : INFO : optimized alpha [0.2287178, 0.19310127, 0.28374416, 0.19464396]
2022-04-15 18:20:00,954 : INFO : merging changes from 2000 documents into 

2022-04-15 18:20:17,059 : INFO : topic #0 (0.238): 0.016*"배우" + 0.015*"캐릭터" + 0.013*"아저씨" + 0.013*"작품" + 0.012*"지옥" + 0.011*"연기" + 0.010*"생각" + 0.010*"드라마" + 0.009*"사랑" + 0.009*"이야기"
2022-04-15 18:20:17,062 : INFO : topic #1 (0.241): 0.068*"사람" + 0.025*"사랑" + 0.023*"생각" + 0.021*"어른" + 0.020*"행복" + 0.015*"드라마" + 0.013*"세상" + 0.013*"이야기" + 0.013*"마음" + 0.010*"현실"
2022-04-15 18:20:17,065 : INFO : topic #2 (0.348): 0.117*"드라마" + 0.019*"대사" + 0.018*"연기" + 0.017*"지안" + 0.015*"배우" + 0.015*"인생" + 0.009*"연출" + 0.009*"작품" + 0.009*"결말" + 0.009*"생각"
2022-04-15 18:20:17,068 : INFO : topic #3 (0.165): 0.017*"작품" + 0.015*"감독" + 0.011*"영화" + 0.010*"캐릭터" + 0.010*"연상호" + 0.008*"드라마" + 0.008*"한국" + 0.007*"인간" + 0.007*"신파" + 0.006*"연민"
2022-04-15 18:20:17,071 : INFO : topic diff=0.407928, rho=0.259605
2022-04-15 18:20:19,016 : INFO : -8.070 per-word bound, 268.8 perplexity estimate based on a held-out corpus of 2000 documents with 30688 words
2022-04-15 18:20:19,017 : INFO : PROGRESS: pass 1, at document 

2022-04-15 18:20:31,715 : INFO : PROGRESS: pass 1, at document #16000/25676
2022-04-15 18:20:32,570 : INFO : optimized alpha [0.25991154, 0.24470142, 0.45624977, 0.20453407]
2022-04-15 18:20:32,577 : INFO : merging changes from 2000 documents into a model of 25676 documents
2022-04-15 18:20:32,584 : INFO : topic #0 (0.260): 0.019*"지옥" + 0.018*"캐릭터" + 0.016*"배우" + 0.012*"작품" + 0.012*"시즌" + 0.011*"연기" + 0.011*"생각" + 0.008*"이야기" + 0.008*"종교" + 0.007*"장면"
2022-04-15 18:20:32,586 : INFO : topic #1 (0.245): 0.047*"사람" + 0.021*"생각" + 0.015*"세상" + 0.014*"인간" + 0.011*"사랑" + 0.011*"이야기" + 0.011*"학교" + 0.009*"사회" + 0.009*"행복" + 0.009*"드라마"
2022-04-15 18:20:32,589 : INFO : topic #2 (0.456): 0.065*"드라마" + 0.027*"연기" + 0.022*"소희" + 0.021*"배우" + 0.015*"연출" + 0.013*"대사" + 0.013*"한소희" + 0.012*"스토리" + 0.012*"액션" + 0.012*"장면"
2022-04-15 18:20:32,592 : INFO : topic #3 (0.205): 0.022*"게임" + 0.016*"작품" + 0.014*"신파" + 0.013*"영화" + 0.011*"감독" + 0.010*"연상호" + 0.010*"한국" + 0.010*"캐릭터" + 0.010*"오징어" + 0.008*"인간"

2022-04-15 18:20:44,203 : INFO : topic diff=0.331968, rho=0.251275
2022-04-15 18:20:45,383 : INFO : -7.899 per-word bound, 238.8 perplexity estimate based on a held-out corpus of 2000 documents with 30688 words
2022-04-15 18:20:45,384 : INFO : PROGRESS: pass 2, at document #4000/25676
2022-04-15 18:20:46,235 : INFO : optimized alpha [0.27253523, 0.31628844, 0.5548747, 0.19436151]
2022-04-15 18:20:46,243 : INFO : merging changes from 2000 documents into a model of 25676 documents
2022-04-15 18:20:46,250 : INFO : topic #0 (0.273): 0.017*"캐릭터" + 0.014*"배우" + 0.012*"시즌" + 0.012*"작품" + 0.010*"지옥" + 0.010*"생각" + 0.010*"이야기" + 0.009*"연기" + 0.008*"작가" + 0.007*"인물"
2022-04-15 18:20:46,252 : INFO : topic #1 (0.316): 0.066*"사람" + 0.022*"생각" + 0.022*"사랑" + 0.017*"어른" + 0.016*"행복" + 0.014*"이야기" + 0.013*"세상" + 0.012*"마음" + 0.011*"인간" + 0.009*"현실"
2022-04-15 18:20:46,255 : INFO : topic #2 (0.555): 0.103*"드라마" + 0.023*"연기" + 0.019*"배우" + 0.018*"대사" + 0.011*"인생" + 0.011*"캐릭터" + 0.011*"연출" + 0.010*"작품" 

...

2022-04-15 18:26:42,021 : INFO : -7.068 per-word bound, 134.2 perplexity estimate based on a held-out corpus of 2000 documents with 20074 words
2022-04-15 18:26:42,024 : INFO : PROGRESS: pass 17, at document #24000/25676
2022-04-15 18:26:42,670 : INFO : optimized alpha [0.5502798, 0.8726455, 1.7471293, 0.5996752]
2022-04-15 18:26:42,676 : INFO : merging changes from 2000 documents into a model of 25676 documents
2022-04-15 18:26:42,684 : INFO : topic #0 (0.550): 0.034*"지옥" + 0.027*"시즌" + 0.017*"이병헌" + 0.016*"폭력" + 0.014*"종교" + 0.008*"박정민" + 0.007*"유아인" + 0.007*"메시지" + 0.006*"시대" + 0.006*"이름"
2022-04-15 18:26:42,687 : INFO : topic #1 (0.873): 0.072*"사람" + 0.028*"사랑" + 0.020*"인간" + 0.019*"생각" + 0.017*"아저씨" + 0.017*"어른" + 0.017*"세상" + 0.016*"마음" + 0.016*"인생" + 0.015*"행복"
2022-04-15 18:26:42,691 : INFO : topic #2 (1.747): 0.094*"드라마" + 0.023*"연기" + 0.022*"대사" + 0.021*"배우" + 0.021*"캐릭터" + 0.018*"작품" + 0.017*"생각" + 0.014*"장면" + 0.014*"연출" + 0.011*"감독"
2022-04-15 18:26:42,694 : INFO : topic #

2022-04-15 18:26:52,386 : INFO : topic #3 (0.579): 0.049*"게임" + 0.014*"오징어" + 0.014*"작품" + 0.013*"한국" + 0.012*"사회" + 0.012*"장르" + 0.010*"넷플릭스" + 0.009*"영화" + 0.009*"자극" + 0.008*"신파"
2022-04-15 18:26:52,389 : INFO : topic diff=0.126292, rho=0.177226
2022-04-15 18:26:53,333 : INFO : -7.654 per-word bound, 201.4 perplexity estimate based on a held-out corpus of 2000 documents with 25795 words
2022-04-15 18:26:53,334 : INFO : PROGRESS: pass 18, at document #12000/25676
2022-04-15 18:26:53,893 : INFO : optimized alpha [0.5408505, 0.835457, 1.7259998, 0.6127367]
2022-04-15 18:26:53,900 : INFO : merging changes from 2000 documents into a model of 25676 documents
2022-04-15 18:26:53,908 : INFO : topic #0 (0.541): 0.017*"시즌" + 0.015*"폭력" + 0.011*"이병헌" + 0.010*"지옥" + 0.010*"김혜자" + 0.009*"역사" + 0.009*"나라" + 0.008*"시대" + 0.008*"혜자" + 0.007*"조선"
2022-04-15 18:26:53,911 : INFO : topic #1 (0.835): 0.065*"사람" + 0.026*"사랑" + 0.019*"생각" + 0.017*"인생" + 0.016*"마음" + 0.016*"행복" + 0.014*"세상" + 0.012*"어른" + 

2022-04-15 18:27:02,494 : INFO : topic #2 (1.752): 0.094*"드라마" + 0.023*"연기" + 0.022*"대사" + 0.021*"배우" + 0.021*"캐릭터" + 0.018*"작품" + 0.017*"생각" + 0.015*"장면" + 0.014*"연출" + 0.011*"감독"
2022-04-15 18:27:02,498 : INFO : topic #3 (0.605): 0.019*"게임" + 0.016*"사회" + 0.016*"작품" + 0.015*"연상호" + 0.012*"신파" + 0.012*"넷플릭스" + 0.011*"한국" + 0.011*"장르" + 0.010*"영화" + 0.010*"문제"
2022-04-15 18:27:02,501 : INFO : topic diff=0.157533, rho=0.177226
2022-04-15 18:27:03,197 : INFO : -6.784 per-word bound, 110.2 perplexity estimate based on a held-out corpus of 1676 documents with 12349 words
2022-04-15 18:27:03,200 : INFO : PROGRESS: pass 18, at document #25676/25676
2022-04-15 18:27:03,589 : INFO : optimized alpha [0.53095996, 0.929087, 1.7077514, 0.5795697]
2022-04-15 18:27:03,595 : INFO : merging changes from 1676 documents into a model of 25676 documents
2022-04-15 18:27:03,601 : INFO : topic #0 (0.531): 0.033*"지옥" + 0.025*"시즌" + 0.017*"폭력" + 0.016*"이병헌" + 0.013*"종교" + 0.008*"박정민" + 0.007*"유아인" + 0.007*"이름

2022-04-15 18:27:12,063 : INFO : topic #1 (0.838): 0.065*"사람" + 0.026*"사랑" + 0.019*"생각" + 0.018*"인생" + 0.016*"마음" + 0.016*"행복" + 0.014*"세상" + 0.012*"어른" + 0.011*"하루" + 0.011*"인간"
2022-04-15 18:27:12,065 : INFO : topic #2 (1.730): 0.079*"드라마" + 0.025*"연기" + 0.024*"배우" + 0.023*"캐릭터" + 0.017*"생각" + 0.017*"대사" + 0.016*"장면" + 0.016*"작품" + 0.015*"연출" + 0.012*"마지막"
2022-04-15 18:27:12,068 : INFO : topic #3 (0.618): 0.035*"게임" + 0.014*"신파" + 0.013*"작품" + 0.013*"한국" + 0.013*"사회" + 0.013*"장르" + 0.012*"학교" + 0.011*"오징어" + 0.011*"자극" + 0.010*"넷플릭스"
2022-04-15 18:27:12,070 : INFO : topic diff=0.188114, rho=0.174507
2022-04-15 18:27:12,862 : INFO : -7.569 per-word bound, 189.9 perplexity estimate based on a held-out corpus of 2000 documents with 17436 words
2022-04-15 18:27:12,863 : INFO : PROGRESS: pass 19, at document #14000/25676
2022-04-15 18:27:13,453 : INFO : optimized alpha [0.53866345, 0.81304055, 1.7799608, 0.6393836]
2022-04-15 18:27:13,460 : INFO : merging changes from 2000 documents into

2022-04-15 18:27:23,159 : INFO : topic #0 (0.531): 0.033*"지옥" + 0.025*"시즌" + 0.017*"폭력" + 0.016*"이병헌" + 0.013*"종교" + 0.008*"박정민" + 0.007*"이름" + 0.007*"유아인" + 0.007*"메시지" + 0.007*"화살촉"
2022-04-15 18:27:23,161 : INFO : topic #1 (0.929): 0.076*"사람" + 0.025*"사랑" + 0.021*"어른" + 0.021*"아저씨" + 0.018*"인생" + 0.018*"생각" + 0.018*"인간" + 0.017*"행복" + 0.016*"마음" + 0.016*"세상"
2022-04-15 18:27:23,164 : INFO : topic #2 (1.711): 0.105*"드라마" + 0.023*"연기" + 0.021*"대사" + 0.021*"배우" + 0.020*"캐릭터" + 0.019*"작품" + 0.017*"생각" + 0.014*"장면" + 0.013*"연출" + 0.012*"이야기"
2022-04-15 18:27:23,167 : INFO : topic #3 (0.584): 0.019*"게임" + 0.017*"사회" + 0.016*"작품" + 0.014*"연상호" + 0.012*"신파" + 0.011*"넷플릭스" + 0.011*"한국" + 0.010*"장르" + 0.010*"문제" + 0.010*"영화"
2022-04-15 18:27:23,170 : INFO : topic diff=0.117206, rho=0.174507
2022-04-15 18:27:23,192 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=9028, num_topics=4, decay=0.5, chunksize=2000) in 468.18s', 'datetime': '2022-04-15T18:27:23.192801', 'gensim':

In [15]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

2022-04-15 18:27:39,900 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2022-04-15 18:27:39,913 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2022-04-15 18:27:39,929 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2022-04-15 18:27:39,947 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2022-04-15 18:27:39,962 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2022-04-15 18:27:39,990 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2022-04-15 18:27:40,018 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2022-04-15 18:27:40,040 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2022-04-15 18:27:40,061 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2022-04-15 18:27:40,080 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2022-04-15 18:27:40,108 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2022-04-15 18:27:40

Average topic coherence: -3.2291.
[([(0.10471779, '드라마'),
   (0.023051577, '연기'),
   (0.021149863, '대사'),
   (0.020926178, '배우'),
   (0.020246599, '캐릭터'),
   (0.018738464, '작품'),
   (0.01679082, '생각'),
   (0.014471733, '장면'),
   (0.013483906, '연출'),
   (0.011537948, '이야기'),
   (0.010793281, '마지막'),
   (0.010442243, '감독'),
   (0.010110605, '느낌'),
   (0.010037513, '정도'),
   (0.010003222, '현실'),
   (0.009311659, '스토리'),
   (0.0075154672, '기대'),
   (0.0070537743, '부분'),
   (0.0069722445, '작가'),
   (0.0068899547, '매력')],
  -2.189812055929553),
 ([(0.018609596, '게임'),
   (0.016501972, '사회'),
   (0.015892196, '작품'),
   (0.014407201, '연상호'),
   (0.011692213, '신파'),
   (0.011429536, '넷플릭스'),
   (0.010738567, '한국'),
   (0.01024241, '장르'),
   (0.009940374, '문제'),
   (0.009762504, '영화'),
   (0.009415802, '인간'),
   (0.00936915, '자극'),
   (0.008335626, '오징어'),
   (0.00828265, '감독'),
   (0.007233473, '학교'),
   (0.006935305, '세계'),
   (0.00657659, '주제'),
   (0.0057260673, '상황'),
   (0.005010325, '비판')

In [16]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [17]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis)

C:\Users\sec\anaconda3\envs\nlp\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
